# 常规赛：遥感影像地块分割gaojiahao899

### 环境安装

In [1]:
!pip install paddlex -i https://mirror.baidu.com/pypi/simple
!pip install imgaug -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 522kB 19.6MB/s eta 0:00:01
     |████████████████████████████████| 102kB 67.9MB/s ta 0:00:01
     |████████████████████████████████| 153kB 27.5MB/s eta 0:00:01
     |████████████████████████████████| 215kB 24.7MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 32.2MB/s eta 0:00:01
     |████████████████████████████████| 102kB 29.9MB/s ta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278362 sha256=c235a5cd1b1c410fc8c256bdae9645337a52339edf8bd292dd5888ae7bf6bddc
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools
  Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4
Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████|

In [4]:
# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
import matplotlib
import os
import paddlex as pdx

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


### 解压数据集

In [ ]:
!unzip -q data/data80164/train_and_label.zip
!unzip -q data/data80164/img_test.zip

### 数据处理
项目使用**PaddleX**进行数据处理和训练:
* **RandomHorizontalFlip** 以一定的概率对图像进行水平翻转。
* **Resize** 调整图像大小。
* **RandomPaddingCrop** 对图像和标注图进行随机裁剪，当所需要的裁剪尺寸大于原图时，则进行padding操作，模型训练时的数据增强操作。
* **RandomBlur** 以一定的概率对图像进行高斯模糊。
* **RandomRotate** 对图像进行随机旋转，模型训练时的数据增强操作。目前支持多通道的RGB图像，例如支持多张RGB图像沿通道轴做concatenate后的图像数据，不支持通道数量不是3的倍数的图像数据。
* **RandomDistort** 以一定的概率对图像进行随机像素内容变换，模型训练时的数据增强操作。目前支持多通道的RGB图像，例如支持多张RGB图像沿通道轴做concatenate后的图像数据，不支持通道数量不是3的倍数的图像数据。
* **Normalize** 对图像进行标准化。

In [5]:
from paddlex.seg import transforms
import imgaug.augmenters as iaa

# 定义训练和验证时的transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(target_size=300),
    transforms.RandomPaddingCrop(crop_size=256),
    transforms.RandomBlur(prob=0.1),
    transforms.RandomRotate(rotate_range=15),
    # transforms.RandomDistort(brightness_range=0.5),
    transforms.Normalize()
])
eval_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.Normalize()
])

In [12]:
import numpy as np

datas = []
image_base = 'img_train'   # 训练集原图路径
annos_base = 'lab_train'   # 训练集标签路径

# 读取原图文件名
ids_ = [v.split('.')[0] for v in os.listdir(image_base)]

# 将训练集的图像集和标签路径写入datas中
for id_ in ids_:
    img_pt0 = os.path.join(image_base, '{}.jpg'.format(id_))
    img_pt1 = os.path.join(annos_base, '{}.png'.format(id_))
    datas.append((img_pt0.replace('/home/aistudio', ''), img_pt1.replace('/home/aistudio', '')))
    if os.path.exists(img_pt0) and os.path.exists(img_pt1):
        pass
    else:
        raise "path invalid!"

# 打印datas的长度和具体存储例子
print('total:', len(datas))
print(datas[0][0])
print(datas[0][1])
print(datas[10][:])

In [5]:
import numpy as np

# 四类标签，这里用处不大，比赛评测是以0、1、2、3类来对比评测的
labels = ['建筑', '耕地', '林地',  '其他']

# 将labels写入标签文件
with open('labels.txt', 'w') as f:
    for v in labels:
        f.write(v+'\n')

# 随机打乱datas
np.random.seed(5)
np.random.shuffle(datas)

# 验证集与训练集的划分，0.05表示5%为训练集，95%为训练集
split_num = int(0.05*len(datas))

# 划分训练集和验证集
train_data = datas[:-split_num]
valid_data = datas[-split_num:]

# 写入训练集list
with open('train_list.txt', 'w') as f:
    for img, lbl in train_data:
        f.write(img + ' ' + lbl + '\n')

# 写入验证集list
with open('valid_list.txt', 'w') as f:
    for img, lbl in valid_data:
        f.write(img + ' ' + lbl + '\n')

# 打印训练集和测试集大小
print('train:', len(train_data))
print('valid:', len(valid_data))



读取语义分割任务数据集，可使用[paddlex.datasets.SegDataset](https://paddlex.readthedocs.io/zh_CN/develop/apis/datasets.html#paddlex-datasets-segdataset)

**paddlex.datasets.SegDataset**参数说明：
* **data_dir (str)**: 数据集所在的目录路径。
* **file_list (str)**: 描述数据集图片文件和对应标注文件的文件路径（文本内每行路径为相对data_dir的相对路径）。
* **label_list (str)**: 描述数据集包含的类别信息文件路径。
* **transforms (paddlex.seg.transforms)**: 数据集中每个样本的预处理/增强算子，详见[paddlex.seg.transforms](https://paddlex.readthedocs.io/zh_CN/develop/apis/transforms/seg_transforms.html#paddlex-seg-transforms)。
* **num_workers (int|str)**：数据集中样本在预处理过程中的线程或进程数。默认为’auto’。当设为’auto’时，根据系统的实际CPU核数设置num_workers: 如果CPU核数的一半大于8，则num_workers为8，否则为CPU核数的一半。
* **buffer_size (int)**: 数据集中样本在预处理过程中队列的缓存长度，以样本数为单位。默认为100。
* **parallel_method (str)**: 数据集中样本在预处理过程中并行处理的方式，支持’thread’线程和’process’进程两种方式。默认为’process’（Windows和Mac下会强制使用thread，该参数无效）。
* **shuffle (bool)**: 是否需要对数据集中样本打乱顺序。默认为False。

In [6]:
data_dir = './'

# 定义训练和验证数据集
train_dataset = pdx.datasets.SegDataset(
    data_dir=data_dir,                # 数据集路径
    file_list='train_list.txt',       # 训练集图片文件list路径
    label_list='labels.txt',          # 训练集标签文件list路径
    transforms=train_transforms,      # train_transforms
    shuffle=True)                     # 数据集是否打乱
    
eval_dataset = pdx.datasets.SegDataset(
    data_dir=data_dir,                # 数据集路径
    file_list='valid_list.txt',       # 验证集图片文件list路径
    label_list='labels.txt',          # 验证集标签文件list路径
    transforms=eval_transforms)       # eval_transforms

### 模型训练

利用[paddlex.seg.DeepLabv3p](https://paddlex.readthedocs.io/zh_CN/develop/apis/models/semantic_segmentation.html)构建DeepLabv3p分割器。

**paddlex.seg.DeepLabv3p**参数说明：
* **num_classes (int)**: 类别数。
* **backbone (str)**: DeepLabv3+的backbone网络，实现特征图的计算，取值范围为[’Xception65’, ‘Xception41’, ‘MobileNetV2_x0.25’, ‘MobileNetV2_x0.5’, ‘MobileNetV2_x1.0’, ‘MobileNetV2_x1.5’, ‘MobileNetV2_x2.0’, ‘MobileNetV3_large_x1_0_ssld’]，默认值为’MobileNetV2_x1.0’。
* **output_stride (int)**: backbone 输出特征图相对于输入的下采样倍数，一般取值为8或16。默认16。
* **aspp_with_sep_conv (bool)**: aspp模块是否采用separable convolutions。默认True。
* **decoder_use_sep_conv (bool)**： decoder模块是否采用separable convolutions。默认True。
* **encoder_with_aspp (bool)**: 是否在encoder阶段采用aspp模块。默认True。
* **enable_decoder (bool)**: 是否使用decoder模块。默认True。
* **use_bce_loss (bool)**: 是否使用bce loss作为网络的损失函数，只能用于两类分割。可与dice loss同时使用。默认False。
* **use_dice_loss (bool)**: 是否使用dice loss作为网络的损失函数，只能用于两类分割，可与bce loss同时使用，当use_bce_loss和use_dice_loss都为False时，使用交叉熵损失函数。默认False。
* **class_weight (list/str)**: 交叉熵损失函数各类损失的权重。当class_weight为list的时候，长度应为num_classes。当class_weight为str时， weight.lower()应为’dynamic’，这时会根据每一轮各类像素的比重自行计算相应的权重，每一类的权重为：每类的比例 * num_classes。class_weight取默认值None是，各类的权重1，即平时使用的交叉熵损失函数。
* **ignore_index (int)**: label上忽略的值，label为ignore_index的像素不参与损失函数的计算。默认255。
* **pooling_crop_size (int)**：当backbone为MobileNetV3_large_x1_0_ssld时，需设置为训练过程中模型输入大小，格式为[W, H]。例如模型输入大小为[512, 512], 则pooling_crop_size应该设置为[512, 512]。在encoder模块中获取图像平均值时被用到，若为None，则直接求平均值；若为模型输入大小，则使用avg_pool算子得到平均值。默认值None。
* **input_channel (int)**: 输入图像通道数。默认值3。

**DeepLabv3p**模型的训练接口，函数内置了**polynomial**学习率衰减策略和**momentum**优化器。

**train**参数说明：
* **num_epochs (int)**: 训练迭代轮数。
* **train_dataset (paddlex.datasets)**: 训练数据读取器。
* **train_batch_size (int)**: 训练数据batch大小。同时作为验证数据batch大小。默认2。
* **eval_dataset (paddlex.datasets)**: 评估数据读取器。
* **save_interval_epochs (int)**: 模型保存间隔（单位：迭代轮数）。默认为1。
* **log_interval_steps (int)**: 训练日志输出间隔（单位：迭代次数）。默认为2。
* **save_dir (str)**: 模型保存路径。默认’output’
* **pretrain_weights (str)**: 若指定为路径时，则加载路径下预训练模型；若为字符串’IMAGENET’，则自动下载在ImageNet图片数据上预训练的模型权重；若为字符串’COCO’，则自动下载在COCO数据集上预训练的模型权重（注意：暂未提供Xception41、MobileNetV2_x0.25、MobileNetV2_x0.5、MobileNetV2_x1.5、MobileNetV2_x2.0的COCO预训练模型）；若为字符串’CITYSCAPES’，则自动下载在CITYSCAPES数据集上预训练的模型权重（注意：暂未提供Xception41、MobileNetV2_x0.25、MobileNetV2_x0.5、MobileNetV2_x1.5、MobileNetV2_x2.0的CITYSCAPES预训练模型）；若为None，则不使用预训练模型。默认’IMAGENET’。
* **optimizer (paddle.fluid.optimizer)**: 优化器。当该参数为None时，使用默认的优化器：使用fluid.optimizer.Momentum优化方法，polynomial的学习率衰减策略。
* **learning_rate (float)**: 默认优化器的初始学习率。默认0.01。
* **lr_decay_power (float)**: 默认优化器学习率衰减指数。默认0.9。
* **use_vdl (bool)**: 是否使用VisualDL进行可视化。默认False。
* **sensitivities_file (str)**: 若指定为路径时，则加载路径下敏感度信息进行裁剪；若为字符串’DEFAULT’，则自动下载在Cityscapes图片数据上获得的敏感度信息进行裁剪；若为None，则不进行裁剪。默认为None。
* **eval_metric_loss (float)**: 可容忍的精度损失。默认为0.05。
* **early_stop (bool)**: 是否使用提前终止训练策略。默认值为False。
* **early_stop_patience (int)**: 当使用提前终止训练策略时，如果验证集精度在early_stop_patience个epoch内连续下降或持平，则终止训练。默认值为5。
* **resume_checkpoint (str)**: 恢复训练时指定上次训练保存的模型路径。若为None，则不会恢复训练。默认值为None。

In [7]:
# 分割类别数
num_classes = len(train_dataset.labels)

# 构建DeepLabv3p分割器
model = pdx.seg.DeepLabv3p(
    num_classes=num_classes,  backbone='Xception65', use_bce_loss=False
)

# 模型训练
model.train(
    num_epochs=50,                 # 训练迭代轮数
    train_dataset=train_dataset,  # 训练集读取
    train_batch_size=8,           # 训练时批处理图片数
    eval_dataset=eval_dataset,    # 验证集读取
    learning_rate=0.0002,         # 学习率
    save_interval_epochs=1,       # 保存模型间隔轮次  
    save_dir='output/deeplab',    # 模型保存路径
    log_interval_steps=200,       # 日志打印间隔
    pretrain_weights='output/deeplab/best_model')  #加载预训练模型

### 模型评估
**evaluate**参数说明：
* **eval_dataset (paddlex.datasets)**: 评估数据读取器。
* **batch_size (int)**: 评估时的batch大小。默认1。
* **epoch_id (int)**: 当前评估模型所在的训练轮数。
* **return_details (bool)**: 是否返回详细信息。默认False。

In [8]:
# 加载模型
model = pdx.load_model('./output/deeplab/best_model')

# 模型评估
model.evaluate(eval_dataset, batch_size=1, epoch_id=None, return_details=False)

### 模型预测
**DeepLabv3p**模型预测接口。需要注意的是，只有在训练过程中定义了eval_dataset，模型在保存时才会将预测时的图像处理流程保存在DeepLabv3p.test_transforms和DeepLabv3p.eval_transforms中。如未在训练时定义eval_dataset，那在调用预测predict接口时，用户需要再重新定义test_transforms传入给predict接口。

**predict**参数说明：

* **img_file (str|np.ndarray)**: 预测图像路径或numpy数组(HWC排列，BGR格式)。
* **transforms (paddlex.seg.transforms)**: 数据预处理操作。

In [9]:
from tqdm import tqdm
import cv2

test_base = 'img_testA/'    # 测试集路径
out_base = 'result/'        # 预测结果保存路径

# 是否存在结果保存路径，如不存在，则创建该路径
if not os.path.exists(out_base):
    os.makedirs(out_base)

# 模型预测并保存预测图片
for im in tqdm(os.listdir(test_base)):
    if not im.endswith('.jpg'):
        continue
    pt = test_base + im
    result = model.predict(pt)
    cv2.imwrite(out_base+im.replace('jpg', 'png'), result['label_map'])

### 其他改进

**跑50个epoch**


In [13]:
# 由预测结果生成提交文件
!zip -r result.zip result/

代码参考：常规赛：遥感影像地块分割baseline